# Capítulo 04 - Metadados e Snapshots

## 📋 Objetivo

Neste capítulo exploraremos a gestão de metadados do Iceberg:
1. Consultar metadados de arquivos (`iceberg_metadata`)
2. Analisar histórico de snapshots (`iceberg_snapshots`)
3. Estatísticas de armazenamento
4. Visualizar evolução da tabela

## 🔧 Requisitos

- DuckDB com extensão Iceberg
- Tabela Iceberg com histórico (ou simulada)

## Setup Inicial

In [8]:
import duckdb
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pyiceberg.catalog.sql import SqlCatalog

# Configurar pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Imports carregados")

✅ Imports carregados


In [9]:
def safe_install_ext(con, ext_name):
    # Dummy function or real one - keeping for compatibility
    pass

# Identificar tabela do capítulo anterior
WAREHOUSE_PATH = './iceberg_warehouse'
CATALOG_DB = f"{WAREHOUSE_PATH}/iceberg_catalog.db"

# Init Catalog (Robust for Windows)
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{CATALOG_DB}",
        "warehouse": f"file://{os.path.abspath(WAREHOUSE_PATH)}",
    },
)

# Verify table exists
try:
    tbl = catalog.load_table("default.sales")
    print(f"✅ Tabela encontrada: default.sales")
    latest_metadata = tbl.metadata_location
    print(f"   Metadata: {os.path.basename(latest_metadata)}")
except Exception as e:
    print(f"⚠️  Tabela não encontrada: {e}")
    latest_metadata = None
    print("   Execute Capítulos 01/03 primeiro")

⚠️  Tabela não encontrada: Table does not exist: default.sales
   Execute Capítulos 01/03 primeiro


## 1. Explorar Metadados de Arquivos

A função `iceberg_metadata()` retorna informações sobre os arquivos de dados (manifests, data files) que compõem a tabela.

In [10]:
def explore_metadata(metadata_file):
    try:
        tbl = catalog.load_table("default.sales")
        
        # PyIceberg inspect.files() returns Arrow Table
        files_table = tbl.inspect.files()
        
        con = duckdb.connect()
        query = """
            SELECT
                file_path,
                record_count,
                file_size_in_bytes / 1024.0 / 1024.0 as size_mb,
                file_format,
                null_value_counts
            FROM files_table
            LIMIT 5
        """
        df = con.execute(query).df()
        return df
    except Exception as e:
        print(f"❌ Erro ao ler metadata: {e}")
        return None

# Executar apenas se tivermos metadata real
if latest_metadata:
    df_meta = explore_metadata(latest_metadata)
    if df_meta is not None and not df_meta.empty:
        print("📋 Metadados de Arquivos:")
        print(df_meta)
    else:
        print("ℹ️  Nenhuma tabela de dados encontrada")
else:
    print("⏩ Pulando execução (sem metadata)")

⏩ Pulando execução (sem metadata)


## 2. Histórico de Snapshots

A função `iceberg_snapshots()` mostra o histórico de commits da tabela, permitindo ver a evolução temporal.

In [11]:
def show_snapshots(metadata_file):
    try:
        tbl = catalog.load_table("default.sales")
        snapshots = tbl.metadata.snapshots
        
        data = []
        for s in snapshots:
            data.append({
                'snapshot_id': s.snapshot_id,
                'timestamp_ms': s.timestamp_ms,
                'sequence_number': s.sequence_number,
                'manifest_list': s.manifest_list
            })
            
        con = duckdb.connect()
        df_source = pd.DataFrame(data)
        
        df = con.execute(f"""
            SELECT
                snapshot_id,
                to_timestamp(timestamp_ms / 1000) as snapshot_time,
                sequence_number,
                manifest_list
            FROM df_source
            ORDER BY sequence_number DESC
        """).df()
        return df
    except Exception as e:
        print(f"❌ Erro ao ler snapshots: {e}")
        return None

if latest_metadata:
    snapshots = show_snapshots(latest_metadata)
    if snapshots is not None and not snapshots.empty:
        print("📸 Histórico de Snapshots:")
        print(snapshots[['snapshot_id', 'snapshot_time', 'sequence_number']])
    else:
        print("ℹ️  Nenhum snapshot encontrado")
else:
    print("⏩ Pulando snapshots (sem metadata)")

⏩ Pulando snapshots (sem metadata)


## 3. Classe Exploradora de Metadados

Vamos criar uma classe reutilizável para encapsular essas análises.

In [12]:
class IcebergMetadataExplorer:
    def __init__(self, metadata_path):
        self.metadata_path = metadata_path
        self.con = duckdb.connect()
        try:
            self.tbl = catalog.load_table("default.sales")
        except:
            self.tbl = None

    def get_stats(self):
        """Estatísticas agregadas da tabela"""
        if not self.tbl:
            return {'files': 0, 'records': 0, 'size_mb': 0}
            
        try:
            files_table = self.tbl.inspect.files()
            
            result = self.con.execute(f"""
                SELECT
                    count(*) as total_files,
                    sum(record_count) as total_records,
                    sum(file_size_in_bytes) / 1024.0 / 1024.0 as size_mb
                FROM files_table
            """).fetchone()
            
            return {
                'files': result[0] or 0,
                'records': result[1] or 0,
                'size_mb': result[2] or 0.0
            }
        except Exception as e:
            print(f"Erro stats: {e}")
            return {'files': 0, 'records': 0, 'size_mb': 0}

    def plot_growth(self):
        """Plota evolução dos snapshots"""
        if not self.tbl: return

        try:
            snapshots = self.tbl.metadata.snapshots
            data = [{'timestamp_ms': s.timestamp_ms, 'sequence_number': s.sequence_number} for s in snapshots]
            df_source = pd.DataFrame(data)

            df = self.con.execute(f"""
                SELECT
                    to_timestamp(timestamp_ms / 1000) as time,
                    sequence_number
                FROM df_source
                ORDER BY sequence_number
            """).df()
            
            if df.empty:
                print("Sem dados para plotar")
                return
                
            plt.figure(figsize=(10, 5))
            plt.plot(df['time'], df['sequence_number'], marker='o')
            plt.title('Evolução dos Snapshots Iceberg')
            plt.xlabel('Data/Hora')
            plt.ylabel('Sequence Number')
            plt.grid(True)
            plt.show()
            
        except Exception as e:
            print(f"Erro plot: {e}")

# Demo Usage
if latest_metadata:
    explorer = IcebergMetadataExplorer(latest_metadata)
    stats = explorer.get_stats()
    print("📊 Estatísticas via Classe:")
    print(stats)
    explorer.plot_growth()

## ✅ Resumo

**Aprendemos:**
1. ✅ Como usar `iceberg_metadata()` para inspecionar arquivos Parquet subjacentes
2. ✅ Como usar `iceberg_snapshots()` para ver auditoria e histórico
3. ✅ Como criar classes para encapsular monitoramento

**Nota:**
- Em produção, monitorar o crescimento de snapshots é vital para decidir quando rodar procedimentos de manutenção (expire_snapshots, rewrite_data_files) que veremos em capítulos futuros.

**Próximo:** Capítulo 05 - Time Travel e Versionamento